In [1]:
import requests
import pandas as pd
import gspread
import df2gspread as d2g
import re

In [2]:


def remove_tags(text):
    TAG_RE = re.compile(r'<[^>]+>')
    tagless_text = TAG_RE.sub(' ', text)
    spaced_text = re.sub(' +', ' ', tagless_text)
    return spaced_text
    

In [3]:
def split_text(text, split_on = ['Characteristics \n','\n Details \n', '\n Roasting \n']):
    return re.split('Characteristics \n|\n Details \n|\n Roasting \n',text)

In [4]:
class ShopifyScraper():
    def __init__(self,root_domain):
        self.domain_url = root_domain
        self.product_list_url = self.domain_url + '/products.json'
        self.product_dict = {}
        
    def get_products(self):
        self.fetch_products = requests.get(self.product_list_url)
        products = self.fetch_products.json()
        #for product in products['products']:
        #    print(product)
        #    print()
       
        for product in products['products']:
            for variant in product['variants']:
                
                #print(variant['title'])

                product_dict_detailed = {}
                product_dict_detailed['product'] = product['title']
                product_dict_detailed['slug'] = product['handle']
                product_dict_detailed['product_type'] = product['product_type']
                product_dict_detailed['beans/nibs'] = variant['option1']
                product_dict_detailed['raw/roasted'] = variant['option2']
                product_dict_detailed['size (lbs)'] = variant['option3']
                product_dict_detailed['size (grams)'] = variant['grams']
                product_dict_detailed['beans/nibs'] = variant['option1']
                product_dict_detailed['available'] = variant['available']
                product_dict_detailed['price'] = variant['price']
                product_dict_detailed['product_id'] = variant['product_id']
                product_dict_detailed['update_date'] = variant['updated_at']
                
                
                
                tagless = remove_tags(product['body_html'])
                '''
                product_dict_detailed['characteristics'] = tagless
                
                
                '''
                splits = split_text(tagless)
                #print(splits)
                try:
                    product_dict_detailed['characteristics'] = splits[1]
                    product_dict_detailed['details'] = splits[2]
                    product_dict_detailed['roasting'] = splits[3]
                except:
                    product_dict_detailed['characteristics'] = tagless
                    product_dict_detailed['details'] = tagless
                    product_dict_detailed['roasting'] = tagless
                #'''
                
                self.product_dict[f"{product['title']} / {variant['title']}"] = product_dict_detailed

    def print_products(self):
        for product in self.product_dict:
            print(product)

In [5]:
x = ShopifyScraper("https://chocolatealchemy2.myshopify.com")
x.get_products()
#x.print_products()

In [6]:
#x.product_dict

In [7]:
ChocolateAlchemy_Products = pd.DataFrame(x.product_dict)
ChocolateAlchemy_Products = ChocolateAlchemy_Products.transpose()
ChocolateAlchemy_Beans = ChocolateAlchemy_Products[(ChocolateAlchemy_Products['product_type'] == 'Beans') & (ChocolateAlchemy_Products['available'] == True)]

In [8]:
ChocolateAlchemy_Beans

product  \
Tanzania - Direct trade/Organic - 2019 / Whole ...  Tanzania - Direct trade/Organic - 2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  Tanzania - Direct trade/Organic - 2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  Tanzania - Direct trade/Organic - 2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  Tanzania - Direct trade/Organic - 2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  Tanzania - Direct trade/Organic - 2019   
...                                                                                    ...   
Ivory Coast - RFA - 2016 / Nibs / Raw / 10 lb                     Ivory Coast - RFA - 2016   
Ivory Coast - RFA - 2016 / Nibs / Raw / 25 lb                     Ivory Coast - RFA - 2016   
Ivory Coast - RFA - 2016 / Whole Beans / Roaste...                Ivory Coast - RFA - 2016   
Ivory Coast - RFA - 2016 / Nibs / Roasted / 1 b...                Ivory Coast - RFA - 2016   
Ivory Coast - RFA - 2016 / Nibs / Raw / 1 bag (...                Ivory Coast - RFA - 2016   

                                                                          slug  \
Tanzania - Direct trade/Organic - 2019 / Whole ...  tanzania-direct-trade-2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  tanzania-direct-trade-2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  tanzania-direct-trade-2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  tanzania-direct-trade-2019   
Tanzania - Direct trade/Organic - 2019 / Whole ...  tanzania-direct-trade-2019   
...                                                                        ...   
Ivory Coast - RFA - 2016 / Nibs / Raw / 10 lb             ivory-coast-rfa-2016   
Ivory Coast - RFA - 2016 / Nibs / Raw / 25 lb             ivory-coast-rfa-2016   
Ivory Coast - RFA - 2016 / Whole Beans / Roaste...        ivory-coast-rfa-2016   
Ivory Coast - RFA - 2016 / Nibs / Roasted / 1 b...        ivory-coast-rfa-2016   
Ivory Coast - RFA - 2016 / Nibs / Raw / 1 bag (...        ivory-coast-rfa-2016   

                                                   product_type   beans/nibs  \
Tanzania - Direct trade/Organic - 2019 / Whole ...        Beans  Whole Beans   
Tanzania - Direct trade/Organic - 2019 / Whole ...        Beans  Whole Beans   
Tanzania - Direct trade/Organic - 2019 / Whole ...        Beans  Whole Beans   
Tanzania - Direct trade/Organic - 2019 / Whole ...        Beans  Whole Beans   
Tanzania - Direct trade/Organic - 2019 / Whole ...        Beans  Whole Beans   
...                                                         ...          ...   
Ivory Coast - RFA - 2016 / Nibs / Raw / 10 lb             Beans         Nibs   
Ivory Coast - RFA - 2016 / Nibs / Raw / 25 lb             Beans         Nibs   
Ivory Coast - RFA - 2016 / Whole Beans / Roaste...        Beans  Whole Beans   
Ivory Coast - RFA - 2016 / Nibs / Roasted / 1 b...        Beans         Nibs   
Ivory Coast - RFA - 2016 / Nibs / Raw / 1 bag (...        Beans         Nibs   

                                                   raw/roasted  \
Tanzania - Direct trade/Organic - 2019 / Whole ...         Raw   
Tanzania - Direct trade/Organic - 2019 / Whole ...         Raw   
Tanzania - Direct trade/Organic - 2019 / Whole ...         Raw   
Tanzania - Direct trade/Organic - 2019 / Whole ...         Raw   
Tanzania - Direct trade/Organic - 2019 / Whole ...         Raw   
...                                                        ...   
Ivory Coast - RFA - 2016 / Nibs / Raw / 10 lb              Raw   
Ivory Coast - RFA - 2016 / Nibs / Raw / 25 lb              Raw   
Ivory Coast - RFA - 2016 / Whole Beans / Roaste...     Roasted   
Ivory Coast - RFA - 2016 / Nibs / Roasted / 1 b...     Roasted   
Ivory Coast - RFA - 2016 / Nibs / Raw / 1 bag (...         Raw   

                                                                                        size (lbs)  \
Tanzania - Direct trade/Organic - 2019 / Whole ...                                           10 lb   
Tanzania - 

In [9]:
gc = gspread.oauth()

In [10]:
cacow_sheet = gc.open('Chocolateering')

In [11]:
cacow_sheet.values_clear("'ChocolateAlchemy_beans'!A1:Z500")

{'spreadsheetId': '1GD0CVYTGsd_FEC7GSVhqpDVhZGOXycC8KJK5uDcY6Mg',
 'clearedRange': 'ChocolateAlchemy_beans!A1:Z500'}

In [12]:
cacow_sheet_CA_products = cacow_sheet.worksheet('ChocolateAlchemy_beans')

In [13]:
cacow_sheet_CA_products.update([ChocolateAlchemy_Beans.columns.values.tolist()] + ChocolateAlchemy_Beans.values.tolist())

{'spreadsheetId': '1GD0CVYTGsd_FEC7GSVhqpDVhZGOXycC8KJK5uDcY6Mg',
 'updatedRange': 'ChocolateAlchemy_beans!A1:N234',
 'updatedRows': 234,
 'updatedColumns': 14,
 'updatedCells': 3276}